In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import pandas as pd
import numpy as np
from datasets import Dataset

In [2]:
#模型选择
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-macbert-base")
model = BertForSequenceClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 加载数据集

In [3]:
# 路径

data_path = r'D:\code\competiton\饿了么智慧点餐\意图识别\train_with_zao.txt'#给出的训练集路径

In [4]:
with open(data_path, 'r') as f:
    text,labels = [],[]
    for line in f: 
        line = line.strip().split('\t')
        text.append(line[0].replace('[','').replace(']','').replace('\'',''))
        labels.append(line[1])

In [5]:
labels,text = labels[1:],text[1:]
labels = [int(i) for i in labels]
print(len(labels),len(text))

11287 11287


In [6]:
text

['来碗杨胃的走吧。',
 '天猫精灵要我皮蛋碎肉捏粥。',
 '天猫精灵.',
 '天猫精灵来碗红豆薏米养胃粥吧。',
 '天猫精灵药碗山药青菜养生粥吧。',
 '来碗热乎南瓜杨胃粥。',
 '我想听安尔晨的人生自古谁无情。',
 '14乘以0.4等于多少？',
 '天猫精灵播播放一首邝美云的堆积情感。',
 '天猫精灵你唱个小跳蛙吧。',
 '天猫精灵喜欢人是什么感觉？',
 '天猫精灵木耳炒肉片来一份。',
 '天猫精灵1.9722乘以7.2。',
 '天猫精灵162除以5等于多少？',
 '天猫精灵我你刚刚是被P麦了吗？',
 '天猫精灵，我想听周深的一燃易。',
 '我想看大然坊絮。',
 '天我再边声音到85，声音到85。',
 '天猫精点。😊',
 '天猫精灵来碗番茄面疙瘩汤吧。',
 '天猫精灵歌曲别看我只是一只羊。',
 '来碗热乎的皮带。',
 '天猫精灵来碗茶树菇老鸭汤面。',
 '天猫精灵要个香辣手撕包菜吧。',
 '天猫精灵来份豆角肉丝小炒吧。',
 '什么等于617加45等于几？',
 '天猫精灵2160乘以8。',
 '天猫精灵来份热乎里砂锅丸子，炖小白菜吧。',
 '天猫精灵来份酸锣卜炖老汤。',
 '刘，我在107加10等于多少？',
 '天猫精灵16.8乘25.2等于多少？',
 '我要上大他的城市飞车。',
 '点个红彩苔小草。',
 '天猫精灵1万38万888888881加1万8000亿等于多少？',
 '天猫精灵播放不遍放屁的声音。',
 '🎼天猫精灵啊，我在你说14减7加5等于多少？',
 '黄瓜炒香菇可以带午餐盒吗？',
 '天猫精灵我想放周深的小管我。😊',
 '天猫精灵来碗鸡肉玉米养生粥吧。',
 '天猫精灵请我播放公司的声音。',
 '天猫精灵要碗热乎的皮蛋瘦肉粥吧。',
 '天猫精灵要碗红枣红豆不蟹粥吧。',
 '天猫精灵15减6除以3等于几？',
 '🎼我想听胡彦斌的望客匆风。',
 '天猫精灵要个韭房肉丝小炒吧。',
 '天猫精灵来粉家常小草发财吧。',
 '天猫精灵人是怎么我宝粑的？😊',
 '天猫精灵12乘以6.1等于多少？',
 '天猫精灵关闭米小圈的那个报时。',
 '天猫精灵是阿里巴巴人工智能实验室于2017年7月5日发布的AI智能产品品牌。但。',
 '天猫精灵来碗红枣银耳热胡粥吧。',
 '天猫

In [7]:
# 创建Dataset对象
dataset_dict = {'text': text, 'labels': labels}
dataset = Dataset.from_dict(dataset_dict)

In [8]:
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 11287
})

In [ ]:
# 数据预处理函数

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True,max_length=64)

# 应用tokenize_function到数据集上
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# 设置格式以便PyTorch可以读取
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#columns_to_remove = ['text',  'token_type_ids']
#tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
# 划分测试集和训练集
train_test_datasets =tokenized_datasets.train_test_split( test_size=0.1, shuffle=True, seed=42)



Map:   0%|          | 0/11287 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets[1]

{'labels': tensor(1),
 'input_ids': tensor([ 101, 1921, 4344, 5125, 4130, 6206, 2769, 4649, 6028, 4810, 5489, 2934,
         5114,  511,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
tokenized_datasets[ 'input_ids']

tensor([[ 101, 3341, 4813,  ...,    0,    0,    0],
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        ...,
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        [ 101, 2769, 2682,  ...,    0,    0,    0],
        [ 101, 3779, 5106,  ...,    0,    0,    0]])

In [ ]:
train_test_datasets['train']


Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10158
})

In [ ]:
# 创建DataLoader
train_dataloader = DataLoader(train_test_datasets['train'].shuffle(seed=42), batch_size=128)
test_dataloader = DataLoader(train_test_datasets['test'], batch_size=128)


In [ ]:

# 打印一个批次的数据以验证
for batch in train_dataloader:
    print(batch)
    break

{'labels': tensor([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 1, 1, 1]), 'input_ids': tensor([[ 101, 1921, 4344,  ...,    0,    0,    0],
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        ...,
        [ 101, 1921, 4344,  ...,    0,    0,    0],
        [ 101, 3341, 4813,  ...,    0,    0,    0],
        [ 101, 1921, 4344,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1,

In [ ]:
# 初始化模型、优化器等

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

# 定义计算指标函数
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
device

d:\envs\transformers\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


device(type='cuda')

In [16]:

# 训练循环
num_epochs = 12
train_losses = []
num_steps = 0
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if num_steps % 100 == 0:
            print(f"Step {num_steps}, Loss: {loss.item()}")
        num_steps += 1
    avg_train_loss = total_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    # 保存模型
    model_save_path = f'bert_binary_classification_model{epoch}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    # 验证循环
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)

            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
            true_labels.extend(batch['labels'].cpu().numpy())

    # 计算指标
    metrics = compute_metrics(predictions, true_labels)
    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss}, Accuracy={metrics['accuracy']}, Precision={metrics['precision']}, Recall={metrics['recall']}, F1={metrics['f1']}")


Step 0, Loss: 0.7517115473747253
Model saved to bert_binary_classification_model0.pth
Epoch 1: Train Loss=0.0914695837549516, Accuracy=0.9875996457041629, Precision=0.9833641404805915, Recall=0.9906890130353817, F1=0.987012987012987
Step 100, Loss: 0.01305742934346199
Model saved to bert_binary_classification_model1.pth
Epoch 2: Train Loss=0.016111425076451268, Accuracy=0.9778565101860053, Precision=1.0, Recall=0.9534450651769087, F1=0.9761677788369876
Step 200, Loss: 0.05632255598902702
Model saved to bert_binary_classification_model2.pth
Epoch 3: Train Loss=0.0097840394353625, Accuracy=0.9858281665190434, Precision=0.9943074003795066, Recall=0.9757914338919925, F1=0.9849624060150376
Step 300, Loss: 0.0011288437526673079
Model saved to bert_binary_classification_model3.pth
Epoch 4: Train Loss=0.0080267391696907, Accuracy=0.9867139061116031, Precision=0.9833333333333333, Recall=0.9888268156424581, F1=0.9860724233983287
Model saved to bert_binary_classification_model4.pth
Epoch 5: Train

AttributeError: 'BertForSequenceClassification' object has no attribute 'path'

In [17]:
# 保存模型
model_save_path = 'bert_binary_classification_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to bert_binary_classification_model.pth


In [18]:
model.load_state_dict(torch.load(r'D:\code\competiton\饿了么智慧点餐\意图识别\bert_binary_classification_model.pth', map_location=torch.device('cuda')))
model.to('cuda')

D:\Temp\ipykernel_24492\3432314076.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'D:\code\competiton\饿了么智慧点餐\意图识别\bert_binary_classif

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [19]:

predictions, true_labels = [], []
with torch.no_grad():

    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())
# 计算指标
metrics = compute_metrics(predictions[:len(true_labels)], true_labels)
print(f" Accuracy={metrics['accuracy']}, Precision={metrics['precision']}, Recall={metrics['recall']}, F1={metrics['f1']}")

 Accuracy=0.9867139061116031, Precision=0.9943181818181818, Recall=0.9776536312849162, F1=0.9859154929577465


In [20]:
print(f"Epoch {epoch+1}: Accuracy={metrics['accuracy']}, Precision={metrics['precision']}, Recall={metrics['recall']}, F1={metrics['f1']}")

Epoch 12: Accuracy=0.9867139061116031, Precision=0.9943181818181818, Recall=0.9776536312849162, F1=0.9859154929577465


In [21]:

# 示例输入文本
new_texts = ["I love programming.", "Python is great.", "I dislike bugs.", "Debugging is fun."]

# 数据预处理函数
def tokenize_function_new(examples):
    return tokenizer(examples, padding='max_length', truncation=True, return_tensors='pt')

# 对新文本进行分词和编码
inputs = tokenize_function_new(new_texts)

# 将输入移动到设备（GPU或CPU）
with torch.no_grad():
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()

# 打印预测结果
for text, label in zip(new_texts, predictions):
    print(f"Text: '{text}' -> Label: {label} ({'Positive' if label == 1 else 'Negative'})")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).